# Sherwood Station Power Company BIORAM Example

This notebook demonstrates the BIORAM compliance example using the latest BOOST Python reference implementation with Pydantic v2 features:

## Key Features

- **Schema Compliance**: All entities conform to BOOST core schemas
- **Dynamic Validation**: Uses schema-driven validation that adapts to changes
- **JSON-LD Support**: Proper semantic web compatibility with `@context`, `@type`, `@id`
- **Pydantic v2**: Uses latest Pydantic features for improved validation
- **None Field Handling**: Optional fields properly excluded from validation
- **ID Pattern Compliance**: Follows BOOST ID conventions (`ORG-*`, `TXN-*`, etc.)
- **Comprehensive Validation**: Full validation across all entities


In [ ]:
import sys
import os
import json
from datetime import datetime, date
from pathlib import Path

# Add BOOST Python reference implementation to path
boost_python_path = Path("../../../../drafts/current/reference-implementations/python")
schema_path = Path("../../../../drafts/current/schema")

# Verify paths exist
if not boost_python_path.resolve().exists():
    raise FileNotFoundError(f"Python implementation not found at {boost_python_path.resolve()}")
if not schema_path.resolve().exists():
    raise FileNotFoundError(f"Schema directory not found at {schema_path.resolve()}")

sys.path.insert(0, str(boost_python_path.resolve()))

# Import BOOST client
from boost_client import create_client

print("✅ BOOST Python reference implementation imported successfully")
print(f"✅ Schema directory found at {schema_path.resolve()}")


In [ ]:
import sys
import os
import json

# Navigate up from the current notebook directory to find the project root ('BOOST')
# This makes the import of boost_client possible
try:
    # Get the directory of the current notebook
    notebook_dir = os.path.dirname(__file__)
except NameError:
    # If __file__ is not defined (e.g., in interactive environment), use current working directory
    notebook_dir = os.getcwd()

# Construct the path to the project root by going up the required number of levels
project_root = os.path.abspath(os.path.join(notebook_dir, "../../../.."))

# Add the project root and the specific reference implementation path to sys.path
reference_implementation_path = os.path.join(project_root, "drafts/current/reference-implementations/python")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
if reference_implementation_path not in sys.path:
    sys.path.insert(0, reference_implementation_path)

# Now the import should work correctly
from boost_client import BOOSTClient
from models import OrganizationType

print(f"Successfully imported BOOSTClient from: {project_root}")


In [ ]:
# Initialize BOOST client with schema path
client = create_client(schema_path=str(schema_path))

# Get schema information
schema_info = client.get_schema_info()
print(f"BOOST client initialized")
print(f"Available entities: {schema_info['available_entities']}")
print(f"Schema loader: {schema_info['schema_loader']}")
print(f"Validator: {schema_info['validator']}")

# Get available enum values from schemas
try:
    org_types = client.get_available_enum_values('organization', 'organizationType')
    print("\nOrganization types:", org_types)
except Exception as e:
    print("\nCould not get organization types:", str(e))

try:
    transaction_statuses = client.get_available_enum_values('transaction', 'transactionStatus')
    print("\nTransaction statuses:", transaction_statuses)
except Exception as e:
    print("\nCould not get transaction statuses:", str(e))


In [ ]:
# Add this as a new cell in your notebook to debug the issue
import sys
import os

# First, let's see what type of organization object we're getting
try:
    # Create an organization
    org = client.create_organization(
        organization_id="ORG-DEBUG-001",
        name="Debug Organization", 
        org_type="supplier"
    )
    
    print(f"Organization object type: {type(org)}")
    print(f"Organization object attributes: {dir(org)}")
    print(f"Has model_dump: {hasattr(org, 'model_dump')}")
    print(f"Has dict: {hasattr(org, 'dict')}")
    
    # Try to see what the actual object looks like
    if hasattr(org, 'model_dump'):
        print("Using model_dump:")
        print(org.model_dump())
    elif hasattr(org, 'dict'):
        print("Using dict:")
        print(org.dict())
    else:
        print("Neither model_dump nor dict available")
        
except Exception as e:
    print(f"Error creating organization: {e}")
    import traceback
    traceback.print_exc()

## Create and Validate Organizations

Let's create the Sherwood Station organization using proper BOOST schema validation with Pydantic v2 features:


In [ ]:
# Generate a proper BOOST organization ID
org_id = client.generate_id("organization", "SHERWOOD-STATION")
print(f"Generated organization ID: {org_id}")

# Create organization using BOOST client with schema validation
sherwood = client.create_organization(
    organization_id=org_id,
    name="Sherwood Station Power Company",
    org_type="producer",
    contactEmail="contact@sherwoodstation.com",
    contactPhone="+15551234567",
    regulatedEntityType="producer",
    operationalStatus="active",
    facilityCapacity={
        "annualCapacity": 200000,
        "unit": "GGE"
    },
    establishedDate="2020-01-01",
    website="https://www.sherwoodstation.com",
    primaryGeographicDataId="GEO-SHERWOOD-001"
)
# Validate organization using v2 features
validation = client.validate_entity(sherwood)
print("\nOrganization Validation:")
if validation['valid']:
    print("Organization validated successfully")
else:
    print("Validation errors:", validation['schema_errors'])

# Display the organization data
org_data = sherwood.dict(by_alias=True)
print("\nOrganization data:")
print(json.dumps(org_data, indent=2, default=str))


## Create Supplier Organizations

Let's create the supplier organizations that provide feedstock to Sherwood Station:


In [ ]:
# Create supplier organizations
suppliers = []

supplier_data = [
    {
        "name": "Anderson Valley Lumber",
        "suffix": "AVL-001"
    },
    {
        "name": "Sierra Forest Products",
        "suffix": "SFP-001"
    }
]

for supplier_info in supplier_data:
    supplier_id = client.generate_id("organization", supplier_info["suffix"])
    supplier = client.create_organization(
        organizationId=supplier_id,
        organizationName=supplier_info["name"],
        organizationType="supplier",  # Biomass suppliers
        contactEmail=f"contact@{supplier_info['suffix'].lower()}.com",
        contactPhone="+15551234567",  # No dashes, follows E.164 format
        regulatedEntityType="not_regulated",  # Biomass suppliers aren't regulated parties
        operationalStatus="active",
        primaryGeographicDataId=f"GEO-{supplier_info['suffix']}"  # Reference to supplier location
    )
    suppliers.append(supplier)
    
    # Validate supplier using v2 features
    validation = client.validate_entity(supplier)
    if validation['valid']:
        print(f"Created and validated supplier: {supplier_info['name']} (ID: {supplier_id})")
    else:
        print(f"Validation errors for {supplier_id}:", validation['schema_errors'])

print(f"\nCreated {len(suppliers)} supplier organizations")

# Display supplier data
print("\nSupplier Organizations:")
for supplier in suppliers:
    supplier_data = supplier.dict(by_alias=True)
    print(f"\n{supplier_data['organizationName']}:")
    print(json.dumps(supplier_data, indent=2, default=str))


## Create Transactions

Let's create transactions between suppliers and Sherwood Station:


In [ ]:
# Create one transaction per supplier
transactions = []

# Average mill waste price for contract value calculation ($/tonne)
mill_waste_price = 75.00

for i, supplier in enumerate(suppliers):
    # Generate transaction ID with proper pattern
    txn_id = client.generate_id("transaction", f"2025-Q1-{i+1:03d}")
    
    # Get supplier data
    supplier_data = supplier.dict(by_alias=True)
    
    # Create transaction with proper BOOST fields
    transaction = client.create_transaction(
        transaction_id=txn_id,
        organization_id=supplier_data["organizationId"],  # Biomass supplier
        customer_id=org_data["organizationId"],  # Sherwood Station
        transaction_date="2025-01-15",
        contractValue=37500.00,  # $75/tonne for biomass
        contractCurrency="USD",
        transactionStatus="completed",
        contractTerms="FOB",
        paymentTerms="Net 30",
        GeographicDataId=supplier_data["primaryGeographicDataId"],  # Where biomass is sourced
        expectedDeliveryDate="2025-01-20",
        complianceRequirements=["RSB_Compliance"],  # Using RSB for biomass sustainability
        reportingPeriod="2025-Q1"
    )
    transactions.append(transaction)
    
    # Validate transaction using v2 features
    validation = client.validate_entity(transaction)
    if validation['valid']:
        print(f"✅ Created and validated transaction {txn_id}")
        # Display transaction data using v2 model_dump
        txn_data = transaction.model_dump(by_alias=True)
        print(json.dumps(txn_data, indent=2, default=str))
    else:
        print(f"❌ Validation errors for {txn_id}:", validation['schema_errors'])

print(f"\n📋 Created {len(transactions)} transactions")


In [ ]:
# Export to JSON-LD format
print("📤 Exporting BOOST data to JSON-LD format...")
jsonld_export = client.export_to_jsonld(include_context=True)

# Parse and display summary
export_data = json.loads(jsonld_export)
entities = export_data.get('@graph', [])

print(f"✅ JSON-LD export completed")
print(f"Total entities exported: {len(entities)}")

# Count entities by type
entity_counts = {}
for entity in entities:
    entity_type = entity.get('@type', 'Unknown')
    entity_counts[entity_type] = entity_counts.get(entity_type, 0) + 1

print("\n📊 Exported entities by type:")
for entity_type, count in entity_counts.items():
    print(f"  {entity_type}: {count}")

print(f"\n📋 JSON-LD Context:")
print(json.dumps(export_data.get('@context', {}), indent=2))


In [ ]:
# Run comprehensive validation
print("🔍 Running comprehensive BOOST validation...")
validation_results = client.validate_all()

if validation_results['valid']:
    print("✅ All entities pass BOOST schema and business logic validation!")
    
    # Display validation summary
    print("\n📊 Validation Summary:")
    print(f"Total organizations: {len(client.organizations)}")
    print(f"Total transactions: {len(client.transactions)}")
    print(f"Schema validation: ✅ PASSED")
    print(f"Business logic validation: ✅ PASSED")
    
else:
    print("❌ Validation errors found:")
    for error in validation_results['errors']:
        print(f"  - {error}")
    
    # Display individual entity results
    if 'entity_results' in validation_results:
        for entity_type, results in validation_results['entity_results'].items():
            if not results.get('valid', True):
                print(f"\n❌ {entity_type.title()} validation errors:")
                for error in results.get('errors', []):
                    print(f"    {error}")

# Save the validated data
output_dir = Path("data")
output_dir.mkdir(exist_ok=True)

# Save organization data
org_file = output_dir / "sherwood_station_organization.json"
with open(org_file, 'w') as f:
    json.dump(org_data, f, indent=2, default=str)
print(f"\n✅ Saved organization data: {org_file}")

# Save supplier organizations
suppliers_data = [s.dict(by_alias=True) for s in suppliers]
suppliers_file = output_dir / "supplier_organizations.json"
with open(suppliers_file, 'w') as f:
    json.dump(suppliers_data, f, indent=2, default=str)
print(f"✅ Saved supplier data: {suppliers_file}")

# Save transactions
transactions_data = [t.dict(by_alias=True) for t in transactions]
transactions_file = output_dir / "transactions.json"
with open(transactions_file, 'w') as f:
    json.dump(transactions_data, f, indent=2, default=str)
print(f"✅ Saved transaction data: {transactions_file}")

# Save complete JSON-LD export
jsonld_file = output_dir / "complete_export.jsonld"
with open(jsonld_file, 'w') as f:
    f.write(jsonld_export)
print(f"✅ Saved JSON-LD export: {jsonld_file}")
